In [1]:
import pandas as pd
import numpy as np
import nltk
import spacy
import re

In [49]:
import warnings
warnings.filterwarnings('once')

In [59]:
from tqdm import tqdm

In [24]:
pd.set_option('display.max_colwidth', -1)

In [3]:
df = pd.read_csv('~/twitter_juul/input/userTimelineData.csv',lineterminator='\n', index_col=None)

In [9]:
df_mari = df[df['tweetText'].str.contains("marijuana")] 

In [10]:
len(df_mari)

3074

In [12]:
df_cann = df[df['tweetText'].str.contains("cannabis")]

In [13]:
len(df_cann)

4412

In [14]:
len(df)

1344182

In [15]:
# we have 3074 with cannabis
# and 4412 tweets with cannabis

In [29]:
# lets see the text in them
df_mari['tweetText']
# some prevention andd studies , promtion campaigns :

# Marijuana and the effects on the brain. #marijuana #teensafety #talkearly https://t.co/agKrMroX5p        

# Free every single person locked up on nonviolent marijuana charges &amp; 
# expunge their records. Right now. https://t.co/GkfVWWSiIy 

# Coke and Aurora Cannabis are reportedly in talks to make marijuana-infused drinks https://t.co/Cl7VPxWBEp            

# 1307955    Since 1989, the #WarOnDrugs has killed ⬆️14X as many people/year as the drugs themselves. - Source: https://t.co/4eWf4Ks2lN - #libertarianism #marijuana https://t.co/ivPILP5sJb     


# some users
# 1313039    @superbmarijuana Check dm  
# 1333093    I mean #marijuana https://t.co/jR3QMFG4Vt  
# 1327265    Just in: Canada just legalized marijuana nationwide.   Canadians will be able to buy and smoke recreational marijuana as soon as September 🍃💨 https://t.co/WmZiWl5y9c                                                                                                                       
# So need a way to separate the studies and people actually consuming and reporting on twitetr..?


271        I bet this is Juuls fault too!   2 million US teens are vaping marijuana, survey finds https://t.co/WodJWowy6C #FoxNews #vape # vapefam                                                                                                                                                                       
592        Teens who vape or use hookah are more likely to use marijuana later, study finds | Get Smart About Drugs: https://t.co/vZIOtIwtMo #Vape #JUUL #GNCY #talkearly #talkoften #prevention                                                                                                                         
598        Marijuana caused more damage to teens' brains than alcohol, study finds #marijuana #prevention https://t.co/2Q9W0kMDFj via @usatoday                                                                                                                                                                          
601        Happening soon: SAM’s congressional briefing on

In [31]:
df_cann['tweetText']

## Promotions
#36036: @SnoopDogg @TamarBraxtonHer @ebenet 🚫Only serious contacts only🚫 High quality plug DM for order/inquiry  📩kik: dabbudskingz  📩//wickr: dabbudskingz 📩hangout/WhatsApp +1 724-739-3867 #weed #weeds #weedfeed #thc #weedporn #kief #cbd #bong #IAmCannabis  #bowlpack #pothead #stoned #kush #high #cannabis #shatter https://t.co/hProFTKAow   
# 1327782 : Cobra by @nikocray   Available online at https://t.co/tPyA2O1Nbf  #whelanssmokeshop #nikocray #cray #cobra #snake #snakeart #cobrakai #artglass #glassart #bongbeauties #glassforsale #glass4sale #glasscollector #glasslife #dabs #dabrig #terps #cannabiscup #hightimes #berkeley https://t.co/AEUXn1j6er
##USEr

## Studies:
# 1332163    #YoungCanadians face heightened crash risk after consuming #cannabis, new study finds @mcgillu @CMAJ https://t.co/5Mg2eTw7pN   

## Politics:
#  Justin Trudeau's nanny state strikes again, this time with plain packaging for cannabis, cigars, and cigarettes. Have your voice heard, and tell the government to leave consumers alone, by going to https://t.co/9XJIYpgBRr #Cdnpoli https://t.co/mNVGGibUQk 

## User
#24172: If cannabis is ever legalised in Ireland, I’m going to move to Clare and open a cannabis café called Spliffs of Moher   

623        Almost 1 in 11 students has vaped #cannabis, report says | Get Smart About Drugs: #vaping #marijuana https://t.co/aIhMI5Fz3T                                                                                                                                                                                                                   
10030      Teens aren't just smoking cannabis, study says -- they're eating and vaping it, too - CNN https://t.co/MtD1zIOof1                                                                                                                                                                                                                              
14478      Synthetic cannabinoids, also known as K2 or spice, are a psychoactive substance produced to mimic the effects of cannabis. https://t.co/bqtTKJ3wNb https://t.co/IsWf0Tkhp1                                                                                                                             

In [33]:
df['tweetText']

0          Cake cake cake cake                                                                                                                                                       
1          @SSEEEGE hahahahhahahaha                                                                                                                                                  
2          Talk back to your ideas your to back talk                                                                                                                                 
3          YOU WILL ALWAYS REMEMBER THIS AS THE DAY U ALMOST CAUGHT CAPTAIN JACK SPARROW                                                                                             
4          @Anubis754 crown #juul                                                                                                                                                    
5          @levelsbeats this is fresh as anything. Well done                              

In [34]:
# preprocess the text : stop words,
# get the vocalb for cannabis
# check all teh tweets that have those occurences
# finally plot the timestamp of those versus juul...

In [50]:
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
#         elif token.orth_.startswith('@'):
#             lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

/home/auppal8/.local/lib/python3.6/site-packages/msgpack_numpy.py:184: PendingDeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


In [51]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/auppal8/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
# getting stpwords from nltk
nltk.download('stopwords')
stopword_set = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/auppal8/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2]
#     tokens = [token for token in tokens if token not in stopword_set]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [41]:
df_tweet = (df['tweetText'].str.lower().str.split())

In [44]:
# removing the stop words
df_tweet = df_tweet.apply(lambda x: [item for item in x if item not in stopword_set])

In [45]:
df_tweet = df_tweet.str.join(" ")

In [46]:
# pattern = re.compile('[\W_]+')--
pattern = re.compile(r'^[a-zA-Z0-9_.-]*$', flags=re.IGNORECASE)
# pattern = re.compile('([^\s\w]|_)+')
df_tweet_p = df_tweet.str.replace(pattern,"")

In [47]:
df_tweet_p = df_tweet_p.str.lstrip().str.rstrip()

In [55]:
df_tweet_p = pd.DataFrame(df_tweet_p)

In [60]:
tqdm.pandas()

In [61]:
# vectorised manner
df_tweet_p['tweetText'] = df_tweet_p['tweetText'].progress_apply(prepare_text_for_lda)

100%|██████████| 1344182/1344182 [13:19<00:00, 1682.19it/s]


In [62]:
df_tweet_p['tweetText']

0          []                                                                                                                                      
1          [@sseeege, hahahahhahahaha]                                                                                                             
2          [idea]                                                                                                                                  
3          [always, remember, almost, catch, captain, sparrow]                                                                                     
4          [@anubis754, crown]                                                                                                                     
5          [@levelsbeats, fresh, anything]                                                                                                         
6          [loyalty]                                                                                            

In [63]:
len(df_tweet_p)

1344182

In [64]:
# applying topic modelling on teh whoel dataset fo the tweets
text_dic = df_tweet_p['tweetText'].values

In [66]:
from gensim import corpora
dictionary = corpora.Dictionary(text_dic)
corpus = [dictionary.doc2bow(text) for text in text_dic]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

/home/auppal8/anaconda3/envs/juul/lib/python3.6/site-packages/ipykernel_launcher.py:5: ResourceWarning: unclosed file <_io.BufferedWriter name='corpus.pkl'>
  """


In [67]:
# LDA for 5 topics
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=8)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)

/home/auppal8/anaconda3/envs/juul/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


In [69]:
## allocate more memory to the instance

In [68]:
# with 5 topics --Dont run the cell
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)

/home/auppal8/anaconda3/envs/juul/lib/python3.6/site-packages/ipykernel_launcher.py:3: ResourceWarning: unclosed file <_io.BufferedReader name='corpus.pkl'>
  This is separate from the ipykernel package so we can avoid doing imports until
/home/auppal8/.local/lib/python3.6/site-packages/numpy/matrixlib/defmatrix.py:68: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


OSError: [Errno 12] Cannot allocate memory

In [ ]:
pyLDAvis.display(lda_display)